In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import dask.dataframe as dd
from memory_profiler import memory_usage

In [2]:
# Jupyter Lab cell extensions
%load_ext rpy2.ipython 
%load_ext memory_profiler 

## 3. Downloading the data

In [3]:
# Necessary metadata
article_id = 14096681  
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharedailyrain/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  
files = data["files"]             
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

In [5]:
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

In [6]:
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

## 4. Combining data CSVs

In [7]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [7]:
%%time
%memit
use_cols = ["time", "lat_min", "lat_max", "lon_min", "lon_max", "rain (mm/day)"]
files = glob.glob('figsharedailyrain/*.csv')
files[:] = [x for x in files if "observed" not in x]
df = pd.DataFrame(columns=use_cols)
df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(model=re.findall(r'[^\/|\\]+(?=\.)', file.replace('_daily_rainfall_NSW', ''))[0])
                for file in files)
              )  
df.to_csv("figsharedailyrain/combined_data.csv")

peak memory: 91.42 MiB, increment: 0.28 MiB
Wall time: 6min 17s


In [3]:
%%time
%memit
#Chirag's computer (Linux OS)
use_cols = ["time", "lat_min", "lat_max", "lon_min", "lon_max", "rain (mm/day)"]
files = glob.glob('figsharedailyrain/*.csv')
files[:] = [x for x in files if "observed" not in x]
df = pd.DataFrame(columns=use_cols)
df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(model=re.findall(r'[^\/|\\]+(?=\.)', file.replace('_daily_rainfall_NSW', ''))[0])
                for file in files)
              )  
df.to_csv("figsharedailyrain/combined_data.csv")

peak memory: 152.25 MiB, increment: 0.12 MiB
CPU times: user 9min 6s, sys: 16 s, total: 9min 22s
Wall time: 9min 26s


## 5. Load the combined CSV to memory and perform a simple EDA

### 5.1.1 Loading in chunks

In [7]:
%%time
%%memit
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("figsharedailyrain/combined_data.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2           1932840
ACCESS-ESM1-5        1610700
AWI-ESM-1-1-LR        966420
BCC-CSM2-MR          3035340
BCC-ESM1              551880
CMCC-CM2-HR4         3541230
CMCC-CM2-SR5         3541230
CMCC-ESM2            3541230
CanESM5               551880
EC-Earth3-Veg-LR     3037320
FGOALS-f3-L          3219300
FGOALS-g3            1287720
GFDL-CM4             3219300
GFDL-ESM4            3219300
INM-CM4-8            1609650
INM-CM5-0            1609650
KIOST-ESM            1287720
MIROC6               2070900
MPI-ESM-1-2-HAM       966420
MPI-ESM1-2-HR        5154240
MPI-ESM1-2-LR         966420
MRI-ESM2-0           3037320
NESM3                 966420
NorESM2-LM            919800
NorESM2-MM           3541230
SAM0-UNICON          3541153
TaiESM1              3541230
combined_data       54385075
dtype: int64
peak memory: 9137.07 MiB, increment: 1455.77 MiB
CPU times: user 1min 35s, sys: 7.34 s, total: 1min 42s
Wall time: 1min 45s


### 5.1.2 DASK

In [7]:
%%time
%%memit
ddf = dd.read_csv("figsharedailyrain/combined_data.csv")
print(ddf["model"].value_counts().compute())

combined_data       54385075
MPI-ESM1-2-HR        5154240
TaiESM1              3541230
CMCC-CM2-HR4         3541230
CMCC-CM2-SR5         3541230
CMCC-ESM2            3541230
NorESM2-MM           3541230
SAM0-UNICON          3541153
FGOALS-f3-L          3219300
GFDL-CM4             3219300
GFDL-ESM4            3219300
EC-Earth3-Veg-LR     3037320
MRI-ESM2-0           3037320
BCC-CSM2-MR          3035340
MIROC6               2070900
ACCESS-CM2           1932840
ACCESS-ESM1-5        1610700
INM-CM5-0            1609650
INM-CM4-8            1609650
KIOST-ESM            1287720
FGOALS-g3            1287720
MPI-ESM-1-2-HAM       966420
MPI-ESM1-2-LR         966420
NESM3                 966420
AWI-ESM-1-1-LR        966420
NorESM2-LM            919800
CanESM5               551880
BCC-ESM1              551880
Name: model, dtype: int64
peak memory: 2396.77 MiB, increment: 1411.14 MiB
CPU times: user 2min 31s, sys: 12 s, total: 2min 43s
Wall time: 57.4 s


### 5.2 Description

* We investigated two approaches,first using chunks and second using dask.
* The first approach loads the combined csv file in chuncks of `10_000_000` and serially performs EDA of counting models
* The second approach using dask internally loads in chunks and performs eda parallely in an optimized way.
* The memory usage in both the first and second approach is similar. 
* The CPU time is higher in the dask approach as compared to first approach. Also there is higher difference between CPU time and Wall time in dask than first approach  
* The wall time of second approach using dask is better than the first approach

## 6. Perform a simple EDA in R

Pick an approach to transfer the dataframe from python to R.
* Parquet file
* Feather file (We chose this one)
* Pandas exchange
* Arrow exchange

### Reasons to choose **feather**

The team referred to lecture notes and online resources to compare the four approaches above and determined to choose **Feather** at the end as the best practice. The reasons are listed as follows,

- **Feather** is faster than `parquet file` and `arrow exchange` when writing files since it store the data with lesser serialization and deserialization, leading to a higher I/O speed.
- `parquet file` has the advantages of saving storage memory. However, it is more appropriate to choose **Feather** as this use case requests a faster speed than data storage.
- **Feather** fits well with the R programming language since the API is embedded well for reading and writing data using R.
- Team researched online articles to compare those four approaches according to experiments and benchmarking results. The team concluded **Feather** is the ideal choice. 
- **Feather** works well among Jupyter notebook sessions. It also speeds up the data queries without taking much memory on the disk, and there is no need for any unpacking when loaded the data back into RAM.

### 6.1 Use Feather to transfer the dataframe from Python to R 

In [ ]:
import pyarrow.dataset as ds
import pyarrow as pa
import pyarrow.parquet as pq
import rpy2.rinterface
import rpy2_arrow.pyarrow_rarrow as pyra
import pyarrow.feather as feather

In [ ]:
%%R
library(arrow)
library(dplyr)
library(tidyr)

In [ ]:
%%time
%%memit
dataset = ds.dataset("figsharedailyrain/combined_data.csv", format="csv")
table = dataset.to_table()

In [ ]:
%%time
# Write in feather format
feather.write_feather(table, 'figsharedailyrain/combined_data.feather')

### 6.2 Perform a simple EDA in R

In [ ]:
%%time
%%R

# Calculate how much time it took to read a feather file
start_time <- Sys.time()
r_table <- arrow::read_feather("figsharedailyrain/combined_data.feather")
print(class(r_table))

# Print the different counts of the models 
result <- r_table %>% count(model) 
end_time <- Sys.time()

print(end_time - start_time)
print(result)

In [ ]:
%%R

# Print the different counts of the time
result <- r_table %>% count(time) 
print(result)

### 6.3 Discussion

There is no difference in terms of the counts for models between the EDA results using Python and R. When comparing the running time, it seems like using R is faster than the speed performed using Python. 

### Summary

There are several challenges the team faced when dealing with large data sets. We can only perform limited and simple EDA with a local machine since large-scale data is very costive to visualize and calculate using complex matrics and evaluation. It is a bit frustrating to run every cell in the Jupyter notebook in our local machine as it will take a lot of time and might be stuck for a while. Besides, downloading files and deleting them would not be efficient enough as a repeating process to trigger the whole data processing.   